# Bias field correction

Attempt 1 : use sitk

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # can copy and deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage
import hashlib
import sys
sys.path.insert(0, '../') # path to functions
from cvasl import file_handler as fh # 
from cvasl import mold #
from cvasl import carve
from cvasl.file_handler import Config

In [ ]:
### Let's see how it works on a 2D, then on a nifti 

In [ ]:
config = Config.from_file()
root_mri_directory = config.get_directory('raw_data')
mri_pattern = os.path.join(root_mri_directory, '**/*.gz')
gz_files = glob.glob(mri_pattern, recursive=True)

In [ ]:

def show_me_debiased(image_filename):
    # reading in on sitk.sitkUInt8 may be wrong, but anyways
    inputImage = sitk.ReadImage(image_filename,sitk.sitkUInt8)

    maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

    inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )

    corrector = sitk.N4BiasFieldCorrectionImageFilter();

    numberFittingLevels = 4

    output = corrector.Execute( inputImage, maskImage )

    outputCasted = sitk.Cast(output,sitk.sitkVectorUInt8)
    
    return outputCasted

In [ ]:
two_d_input_string = 'test/N4BiasFieldCorrection_input_image.png'
two_d_input = sitk.ReadImage('test/N4BiasFieldCorrection_input_image.png')
print(two_d_input.GetPixelIDTypeAsString())
print(two_d_input.GetOrigin())
print(two_d_input.GetSpacing())
print(two_d_input.GetDirection())
print(two_d_input.GetNumberOfComponentsPerPixel())

In [ ]:
outputCasted = show_me_debiased('test/N4BiasFieldCorrection_input_image.png')
inputImage = sitk.ReadImage('test/N4BiasFieldCorrection_input_image.png',sitk.sitkUInt8)

In [ ]:
npa = sitk.GetArrayViewFromImage(outputCasted)
opa = sitk.GetArrayViewFromImage(inputImage)
plt.subplot(1, 2, 1) # row 1, col 2 index 1
plt.imshow(npa,cmap='Greys',  interpolation='nearest')
plt.title("Output image")
plt.xlabel('X-axis ')
plt.ylabel('Y-axis ')

plt.subplot(1, 2, 2) # index 2
plt.imshow(opa,cmap='Greys',  interpolation='nearest')
plt.title("Original image")
plt.xlabel('X-axis ')
plt.ylabel('Y-axis ')

plt.show()

In [ ]:
outputCasted = show_me_debiased(gz_files[0])


# Read the .nii image containing the volume with SimpleITK:
t1_fn = gz_files[0]
sitk_t1 = sitk.ReadImage(t1_fn)

# and access the numpy array:
t1 = sitk.GetArrayFromImage(sitk_t1)

In [ ]:
# the code below currently breaks and needs fixing

In [ ]:
mold.SliceViewer(t1)

In [ ]:
#sitk.GetArrayFromImage?

In [ ]:
output = sitk.GetArrayFromImage(outputCasted)

In [ ]:
mold.SliceViewer(output)

So above the data type was wrong, let's recast on a richer data type? Or maybe some other parameter like our mask needs a reset.

In [ ]:
# sitk_t1 

print(sitk_t1.GetPixelIDTypeAsString())
print(sitk_t1.GetOrigin())
print(sitk_t1.GetSpacing())
print(sitk_t1.GetDirection())
print(sitk_t1.GetNumberOfComponentsPerPixel())

In [ ]:

def richer_debiased(image_filename, fitting_levels, bits_in_input):
    # 
    print(type(bits_in_input))
    inputImage = sitk.ReadImage(image_filename,bits_in_input )

    maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

    inputImage = sitk.Cast( inputImage, bits_in_input )

    corrector = sitk.N4BiasFieldCorrectionImageFilter();

    numberFittingLevels = fitting_levels

    output = corrector.Execute( inputImage, maskImage )

    outputCasted = sitk.Cast(output,bits_in_input)
    
    return outputCasted

In [ ]:
output_recasted = richer_debiased(gz_files[0], 40, sitk.sitkFloat32 )


In [ ]:
output2 = sitk.GetArrayFromImage(output_recasted)

In [ ]:
mold.SliceViewer(output2)

Still needs revision to comprehend best masks

In [ ]:

def n4_debias_sitk(image_filename, iteration_vector = [20, 10, 10, 5], masking=True):
    
    # 
    
    inputImage = sitk.ReadImage(image_filename)
    bits_in_input = inputImage.GetPixelIDTypeAsString()
    bit_dictionary = {"Signed 8 bit integer": sitk.sitkInt8,
                      "Signed 16 bit integer": sitk.sitkInt16,
                      "Signed 32 bit integer": sitk.sitkInt32,
                      "Signed 64 bit integer": sitk.sitkInt64,
                      "Unsigned 8 bit integer": sitk.sitkUInt8,
                      "Unsigned 16 bit integer":sitk.sitkUInt16,
                      "Unsigned 32 bit integer":sitk.sitkUInt32,
                      "Unsigned 64 bit integer":sitk.sitkUInt64,
                      "32-bit float": sitk.sitkFloat32,
                      "64-bit float": sitk.sitkFloat64,}
    bits_ing = bit_dictionary[bits_in_input] 
    #print(bits_ing)

    maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

    inputImage = sitk.Cast( inputImage, bits_ing )

    corrector = sitk.N4BiasFieldCorrectionImageFilter();


    
    corrector.SetMaximumNumberOfIterations(iteration_vector)
    
    if masking:

        output = corrector.Execute( inputImage, maskImage )
    else:
        output = corrector.Execute( inputImage )

    outputCasted = sitk.Cast(output,bits_ing)
    
    return outputCasted

In [ ]:
output_recasted2 = n4_debias_sitk(gz_files[0], [20, 10, 10, 5], masking=False )


In [ ]:
output3 = sitk.GetArrayFromImage(output_recasted2)

In [ ]:
mold.SliceViewer(output3)

OK, now we need to do something with the number of fitting levels...
OK, now we need to fix the mask question, and what exactly the mask should be

In [ ]:
?corrector